In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from datetime import datetime
import datetime as dt

import sklearn.linear_model as lm
import sklearn

from IPython.display import display, Markdown, Latex, Math

# COVID-19

## DS100 Final Project

Gene Ho, Hannah Qi, Jasmine Wu

## Research Questions

<!-- OLD QUESTIONS: 1. How does the timing of the implementation of various social distancing measures (i.e. time to issue) impact the growth rate of cases?
2. Who would be considered as a highly susceptible population of COVID-19? What characteristics make people more susceptible to COVID-19? -->

1. How does the timing of the implementation of various social distancing measures (i.e. time to issue) predict a county's democrat to republican ratio?
2. How does the timing of the implementation of various social distancing measures (i.e. time to issue) predict a county's political party majority?

## Data Cleaning

In [24]:
counties = pd.read_csv("./covid-data/abridged_counties.csv")

In [25]:
counties.head()

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,POP_LONGITUDE,...,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,SVIPercentile,HPSAShortage,HPSAServedPop,HPSAUnderservedPop
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,-86.494165,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4354,NaN,NaN,NaN
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,-87.762381,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,-85.310038,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.9959,6.08,5400.0,18241.0
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,-87.127659,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.6003,2.75,14980.0,6120.0
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,-86.591491,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4242,7.21,31850.0,25233.0


In [26]:
counties.columns

Index(['countyFIPS', 'STATEFP', 'COUNTYFP', 'CountyName', 'StateName', 'State',
       'lat', 'lon', 'POP_LATITUDE', 'POP_LONGITUDE', 'CensusRegionName',
       'CensusDivisionName', 'Rural-UrbanContinuumCode2013',
       'PopulationEstimate2018', 'PopTotalMale2017', 'PopTotalFemale2017',
       'FracMale2017', 'PopulationEstimate65+2017',
       'PopulationDensityperSqMile2010', 'CensusPopulation2010',
       'MedianAge2010', '#EligibleforMedicare2018',
       'MedicareEnrollment,AgedTot2017', '3-YrDiabetes2015-17',
       'DiabetesPercentage', 'HeartDiseaseMortality', 'StrokeMortality',
       'Smokers_Percentage', 'RespMortalityRate2014', '#FTEHospitalTotal2017',
       'TotalM.D.'s,TotNon-FedandFed2017', '#HospParticipatinginNetwork2017',
       '#Hospitals', '#ICU_beds', 'dem_to_rep_ratio', 'PopMale<52010',
       'PopFmle<52010', 'PopMale5-92010', 'PopFmle5-92010', 'PopMale10-142010',
       'PopFmle10-142010', 'PopMale15-192010', 'PopFmle15-192010',
       'PopMale20-242010', 'P

In [27]:
confirmed_df = pd.read_csv("./covid-data/time_series_covid19_confirmed_US.csv")
confirmed_df.head(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,128,130,133,133,133,133,135,135,136,136
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,11,11,11,11,11,11,13,13,13,14
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,...,683,725,788,897,903,923,974,1043,1068,1118
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,...,45,50,51,51,51,51,51,51,51,53
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15,17,19,19,19,23,24,26,26,25
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,56,59,66,71,72,87,91,101,103,109
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,4,9,9,10,10,11,12,14,15,18
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,9,11,13,16,17,17,18,22,24,26
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,11,12,12,13,14,16,17,18,20,20


In [28]:
# created a new table
df = counties[[
    'countyFIPS', 'STATEFP', 'COUNTYFP', 'CountyName', 'StateName', 'State',
    'lat', 'lon', 'POP_LATITUDE', 'PopulationEstimate2018', 'POP_LONGITUDE',
    'dem_to_rep_ratio', 'stay at home', '>50 gatherings', '>500 gatherings',
    'public schools', 'restaurant dine-in', 'entertainment/gym',
    'federal guidelines', 'foreign travel ban'
]]

In [29]:
df.head(5)

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,POP_LONGITUDE,dem_to_rep_ratio,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,55601.0,-86.494165,0.326656,737519.0,737504.0,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,218022.0,-87.762381,0.253255,737519.0,737504.0,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,24881.0,-85.310038,0.893106,737519.0,737504.0,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,22400.0,-87.127659,0.278124,737519.0,737504.0,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,57840.0,-86.591491,0.094317,737519.0,737504.0,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0


In [30]:
# from https://gist.github.com/JeffPaine/3083347
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands': 'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

df["State"] = df['StateName'].map(dict(map(reversed, us_state_abbrev.items())))

/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
# a function to convert times from Gregorian ordinal date to pandas Timestamp
def convert_time(df, columns):
    data = df.copy()

    if isinstance(columns, list):
        for col in columns:
            if not isinstance(data[col][0], pd.Timestamp):
                data[col] = data[col].dropna().astype(int).map(
                    pd.Timestamp.fromordinal)
    else:
        if not isinstance(data[columns][0], pd.Timestamp):
            data[columns] = data[columns].dropna().astype(int).map(
                pd.Timestamp.fromordinal)
    return data

In [32]:
# apply func to the dataframe
df = convert_time(df, [
    'stay at home', '>50 gatherings', '>500 gatherings', 'public schools',
    'restaurant dine-in', 'entertainment/gym', 'federal guidelines',
    'foreign travel ban'
])
df

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,POP_LONGITUDE,dem_to_rep_ratio,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,55601.0,-86.494165,0.326656,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,218022.0,-87.762381,0.253255,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,24881.0,-85.310038,0.893106,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,22400.0,-87.127659,0.278124,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,57840.0,-86.591491,0.094317,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,15005,15.0,5.0,Kalawao,HI,Hawaii,NaN,NaN,21.188495,88.0,-156.979972,NaN,2020-03-25,2020-03-25,2020-03-25,2020-03-23,2020-03-20,2020-03-25,2020-03-16,2020-03-11
3240,72039,72.0,39.0,Ciales Municipio,PR,Puerto Rico,NaN,NaN,18.314399,15918.0,-66.494215,NaN,NaT,NaT,NaT,2020-03-16,2020-03-15,2020-03-15,2020-03-16,2020-03-11
3241,72069,72.0,69.0,Humacao Municipio,PR,Puerto Rico,NaN,NaN,18.144804,50532.0,-65.817109,NaN,NaT,NaT,NaT,2020-03-16,2020-03-15,2020-03-15,2020-03-16,2020-03-11
3242,City1,NaN,NaN,New York City,NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [33]:
# dropping NYC and Kansas City because they are cities not counties
df.drop([3242, 3243], inplace=True)
# df[df['CountyName'].isna()]


# NYC is **not** a county so we are expecting an empty df
df[df['CountyName'] == 'New York City']

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,POP_LONGITUDE,dem_to_rep_ratio,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban


In [34]:
df.head()

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,POP_LONGITUDE,dem_to_rep_ratio,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,55601.0,-86.494165,0.326656,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,218022.0,-87.762381,0.253255,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,24881.0,-85.310038,0.893106,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,22400.0,-87.127659,0.278124,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,57840.0,-86.591491,0.094317,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11


In [35]:
# creating a new column "new FIPS" of df's "countyFIPS" as integer
df = df[df['countyFIPS'].notna()]
# df.astype({'countyFIPS':'int32'}).dtypes
df['new FIPS'] = df['countyFIPS'].astype('int32')
df

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,...,dem_to_rep_ratio,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,new FIPS
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,55601.0,...,0.326656,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11,1001
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,218022.0,...,0.253255,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11,1003
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,24881.0,...,0.893106,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11,1005
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,22400.0,...,0.278124,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11,1007
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,57840.0,...,0.094317,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,02230,2.0,230.0,Skagway Municipality,AK,Alaska,NaN,NaN,59.464536,1148.0,...,NaN,2020-03-28,2020-03-24,2020-03-24,2020-03-19,2020-03-17,2020-03-17,2020-03-16,2020-03-11,2230
3238,02275,2.0,275.0,Wrangell City and Borough,AK,Alaska,NaN,NaN,56.385821,2503.0,...,NaN,2020-03-28,2020-03-24,2020-03-24,2020-03-19,2020-03-17,2020-03-17,2020-03-16,2020-03-11,2275
3239,15005,15.0,5.0,Kalawao,HI,Hawaii,NaN,NaN,21.188495,88.0,...,NaN,2020-03-25,2020-03-25,2020-03-25,2020-03-23,2020-03-20,2020-03-25,2020-03-16,2020-03-11,15005
3240,72039,72.0,39.0,Ciales Municipio,PR,Puerto Rico,NaN,NaN,18.314399,15918.0,...,NaN,NaT,NaT,NaT,2020-03-16,2020-03-15,2020-03-15,2020-03-16,2020-03-11,72039


In [36]:
# creating a new column "new FIPS" of confimed_df's "FIP" as integer

confirmed_df = confirmed_df.rename(columns={'Admin2': 'CountyName'})
confirmed_df = confirmed_df[confirmed_df['FIPS'].notna()]
confirmed_df["new FIPS"] = confirmed_df['FIPS'].astype('int32')
confirmed_df

,UID,iso2,iso3,code3,FIPS,CountyName,Province_State,Country_Region,Lat,Long_,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,new FIPS
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,60
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,130,133,133,133,133,135,135,136,136,66
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,11,11,11,11,11,13,13,13,14,69
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,725,788,897,903,923,974,1043,1068,1118,72
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,50,51,51,51,51,51,51,51,53,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,84090053,US,USA,840,90053.0,Unassigned,Washington,US,0.0000,0.0000,...,908,848,732,619,484,383,272,334,444,90053
3249,84090054,US,USA,840,90054.0,Unassigned,West Virginia,US,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,90054
3250,84090055,US,USA,840,90055.0,Unassigned,Wisconsin,US,0.0000,0.0000,...,0,0,0,0,1,1,0,0,0,90055
3251,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0,90056


In [37]:
# joining df table and confirmed_df table by "new FIPS"
joint_table = pd.merge(df, confirmed_df, how='inner', on=['new FIPS'])

In [38]:
## column index of new FIPS and dates (i.e. simplified confirmed df)
# joint_table.columns.get_loc("new FIPS") # 20
# joint_table.columns.get_loc("4/18/20") # 119
# joint_table.columns.get_loc("1/22/20") # 32
another_attempt_first = joint_table.iloc[:, np.r_[20, 32:119]]

another_attempt_first.head()

,new FIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
0,1001,0,0,0,0,0,0,0,0,0,...,12,15,17,19,19,19,23,24,26,26
1,1003,0,0,0,0,0,0,0,0,0,...,44,56,59,66,71,72,87,91,101,103
2,1005,0,0,0,0,0,0,0,0,0,...,3,4,9,9,10,10,11,12,14,15
3,1007,0,0,0,0,0,0,0,0,0,...,9,9,11,13,16,17,17,18,22,24
4,1009,0,0,0,0,0,0,0,0,0,...,10,11,12,12,13,14,16,17,18,20


In [39]:
# function that catches each county's date of first case. if no date exists for that county, its first date will be recorded as 4/19/20
def first_case(df):
    firstcasedate_list = []
    for i in df.index:
        if df.iloc[i, len(df.columns) - 1] == 0:
            firstcasedate_list.append(
                '4/19/20'
            )  #equivalent to NA value since it is not included in the df
            continue
        for j in range(1, len(df.columns)):
            if df.iloc[i, j] > 0:
                date = df.columns[j]
                firstcasedate_list.append(date)
                break
    return firstcasedate_list

In [40]:
# joint table with new column of each county's date of first case
joint_table["Date of First Case"] = pd.Series(
    first_case(another_attempt_first))

joint_table

,countyFIPS,STATEFP,COUNTYFP,CountyName_x,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,Date of First Case
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,55601.0,...,17,19,19,19,23,24,26,26,25,3/24/20
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,218022.0,...,59,66,71,72,87,91,101,103,109,3/15/20
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,24881.0,...,9,9,10,10,11,12,14,15,18,4/3/20
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,22400.0,...,11,13,16,17,17,18,22,24,26,3/30/20
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,57840.0,...,12,12,13,14,16,17,18,20,20,3/25/20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,02195,2.0,195.0,Petersburg Borough,AK,Alaska,NaN,NaN,56.812712,3221.0,...,2,2,2,2,2,2,2,2,2,4/3/20
3136,02198,2.0,198.0,Prince of Wales-Hyder Census Area,AK,Alaska,NaN,NaN,55.448164,6422.0,...,0,1,1,1,2,2,2,2,2,4/11/20
3137,02230,2.0,230.0,Skagway Municipality,AK,Alaska,NaN,NaN,59.464536,1148.0,...,0,0,0,0,0,0,0,0,0,4/19/20
3138,02275,2.0,275.0,Wrangell City and Borough,AK,Alaska,NaN,NaN,56.385821,2503.0,...,0,0,0,0,0,0,0,0,0,4/19/20


In [41]:
# changing the format of date in the joint_table
joint_table["Date of First Case"] = pd.to_datetime(
    joint_table["Date of First Case"]).dt.strftime('%Y-%m-%d')
joint_table.tail()

,countyFIPS,STATEFP,COUNTYFP,CountyName_x,StateName,State,lat,lon,POP_LATITUDE,PopulationEstimate2018,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,Date of First Case
3135,02195,2.0,195.0,Petersburg Borough,AK,Alaska,NaN,NaN,56.812712,3221.0,...,2,2,2,2,2,2,2,2,2,2020-04-03
3136,02198,2.0,198.0,Prince of Wales-Hyder Census Area,AK,Alaska,NaN,NaN,55.448164,6422.0,...,0,1,1,1,2,2,2,2,2,2020-04-11
3137,02230,2.0,230.0,Skagway Municipality,AK,Alaska,NaN,NaN,59.464536,1148.0,...,0,0,0,0,0,0,0,0,0,2020-04-19
3138,02275,2.0,275.0,Wrangell City and Borough,AK,Alaska,NaN,NaN,56.385821,2503.0,...,0,0,0,0,0,0,0,0,0,2020-04-19
3139,15005,15.0,5.0,Kalawao,HI,Hawaii,NaN,NaN,21.188495,88.0,...,0,0,0,0,0,0,0,0,0,2020-04-19


In [42]:
# a new dataframe with more relevant columns
new_df = joint_table[[
    'countyFIPS', 'CountyName_x', 'StateName', 'State', 'dem_to_rep_ratio',
    'Date of First Case', 'stay at home', '>50 gatherings', '>500 gatherings',
    'public schools', 'restaurant dine-in', 'entertainment/gym'
]]
new_df

,countyFIPS,CountyName_x,StateName,State,dem_to_rep_ratio,Date of First Case,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym
0,01001,Autauga,AL,Alabama,0.326656,2020-03-24,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
1,01003,Baldwin,AL,Alabama,0.253255,2020-03-15,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
2,01005,Barbour,AL,Alabama,0.893106,2020-04-03,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
3,01007,Bibb,AL,Alabama,0.278124,2020-03-30,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
4,01009,Blount,AL,Alabama,0.094317,2020-03-25,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...
3135,02195,Petersburg Borough,AK,Alaska,NaN,2020-04-03,2020-03-28,2020-03-24,2020-03-24,2020-03-19,2020-03-17,2020-03-17
3136,02198,Prince of Wales-Hyder Census Area,AK,Alaska,NaN,2020-04-11,2020-03-28,2020-03-24,2020-03-24,2020-03-19,2020-03-17,2020-03-17
3137,02230,Skagway Municipality,AK,Alaska,NaN,2020-04-19,2020-03-28,2020-03-24,2020-03-24,2020-03-19,2020-03-17,2020-03-17
3138,02275,Wrangell City and Borough,AK,Alaska,NaN,2020-04-19,2020-03-28,2020-03-24,2020-03-24,2020-03-19,2020-03-17,2020-03-17


In [43]:
# print(new_df['dem_to_rep_ratio'].isna().sum()) # 26 counties without our outcome

new_df.dropna(subset=['dem_to_rep_ratio'], inplace=True)
#we had to drop all the rows that had NAN in the 'dem_to_rep_ratio' column
#some were counties from Alaska and Hawaii

/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
# some counties did not have a date of order for 'stay at home', '>50 gatherings', '>500 gatherings', and 'entertainment/gym'
# assume that they implemented it after the last day of data collection
# impute with last date + 1 = April 19, 2020
new_df['Date of First Case'] = pd.to_datetime(new_df['Date of First Case'])
# april_19 = dt.datetime.strptime("2020-04-19", '%Y-%m-%d')
april_19 = pd.to_datetime("2020-04-19")

var_columns = [
    'Date of First Case', 'stay at home', '>50 gatherings', '>500 gatherings',
    'public schools', 'restaurant dine-in', 'entertainment/gym'
]

new_df[var_columns].apply(pd.to_datetime, errors='coerce')

print(new_df.isna().sum())

new_df = new_df.fillna(april_19)

countyFIPS              0
CountyName_x            0
StateName               0
State                   0
dem_to_rep_ratio        0
Date of First Case      0
stay at home          489
>50 gatherings        118
>500 gatherings       118
public schools          0
restaurant dine-in      0
entertainment/gym      65
dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
# making sure there are no more NAN dates for the orders
new_df.isna().sum()

countyFIPS            0
CountyName_x          0
StateName             0
State                 0
dem_to_rep_ratio      0
Date of First Case    0
stay at home          0
>50 gatherings        0
>500 gatherings       0
public schools        0
restaurant dine-in    0
entertainment/gym     0
dtype: int64

In [24]:
new_df

,countyFIPS,CountyName_x,StateName,State,dem_to_rep_ratio,Date of First Case,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym
0,01001,Autauga,AL,Alabama,0.326656,2020-03-24,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
1,01003,Baldwin,AL,Alabama,0.253255,2020-03-15,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
2,01005,Barbour,AL,Alabama,0.893106,2020-04-03,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
3,01007,Bibb,AL,Alabama,0.278124,2020-03-30,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
4,01009,Blount,AL,Alabama,0.094317,2020-03-25,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...
3129,56041,Uinta,WY,Wyoming,0.195320,2020-04-02,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19
3130,56043,Washakie,WY,Wyoming,0.182755,2020-03-27,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19
3131,56045,Weston,WY,Wyoming,0.098582,2020-04-19,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19
3132,08014,Broomfield,CO,Colorado,1.373356,2020-03-20,2020-03-26,2020-03-24,2020-03-13,2020-03-23,2020-03-20,2020-03-17


In [0]:
# number of days between date of first case and order (order - date of first case)

for column in var_columns[1:]:
    new_df.loc[:, column] = pd.to_datetime(new_df.loc[:, column])
    new_df[column] = new_df[column] - new_df['Date of First Case']
    new_df[column] = new_df[column].dt.days
#     new_df[column] = new_df[column].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)

In [0]:
new_df.head()

In [0]:
# a final dataframe
final_df = new_df[[
    'countyFIPS', 'CountyName_x', 'StateName', 'dem_to_rep_ratio',
    'Date of First Case', 'stay at home', '>50 gatherings', '>500 gatherings',
    'public schools', 'restaurant dine-in', 'entertainment/gym'
]]

final_df = final_df.rename(columns = {'CountyName_x' : 'CountyName'})

print(final_df.shape[0], "counties in our df")
final_df.head()

# negative values means county was proactive and implemented it before first case

## EDA

### COVID-19 Landscape

### Order of Orders

In [0]:
# order by which order is implemented first

temp = np.flip(
    final_df[[
        'stay at home', '>50 gatherings', '>500 gatherings', 'public schools',
        'restaurant dine-in', 'entertainment/gym'
    ]].values.argsort(), 1)
ndf = pd.DataFrame(final_df[[
    'stay at home', '>50 gatherings', '>500 gatherings', 'public schools',
    'restaurant dine-in', 'entertainment/gym'
]].columns[temp])

# final_df.columns

ndf.head()

In [0]:
orders_order = pd.DataFrame()
for col in ndf.columns:
    orders_order[col] = ndf[col].value_counts()

In [0]:
orders_order = orders_order.fillna(0).rename(columns={
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6
})

orders_order = orders_order.reset_index().melt('index',
                                               var_name='cols',
                                               value_name='vals')

In [0]:
orders_order.head()

In [0]:
# sns.distplot(orders_order.columns, hist_kws={"weights":orders_order.values()})


# not sure if this way makes sense
# can potentially swap "cols" and "index"
order_bar = sns.catplot(x="cols", y="vals", hue='index', data=orders_order, kind = "bar", orient = "v")

# order_bar.set_xticklabels(rotation=90)

plt.title("Frequency of Order of COVID-19 Orders")

order_bar.set_xticklabels(['1st', '2nd', '3rd', '4th', '5th', '6th'])
plt.xlabel("Order Number")
plt.ylabel("Count")
order_bar._legend.set_title("Order")

;

### Outcome Variable: Democrat to Republican Ratio

In [0]:
# summary statistics for our outcome of interest

print(min(final_df['dem_to_rep_ratio']), np.median(final_df['dem_to_rep_ratio']),
      np.mean(final_df['dem_to_rep_ratio']),
      max(final_df['dem_to_rep_ratio']))

In [0]:
# taking a look at the outliers

# to get the 25th percentile/first quartile
Q1 = np.percentile(final_df['dem_to_rep_ratio'], 25)
# to get the 75th percentile/third quartile
Q3 = np.percentile(final_df['dem_to_rep_ratio'], 75)
# to get IQR
dem_rep_iqr = Q3- Q1 #0.461
# to get outliers
outliers_below_Q1 = Q1 - 1.5*dem_rep_iqr # -0.42
outliers_above_Q3 = Q3 + 1.5*dem_rep_iqr # 1.42

# outlier counties
# (final_df[final_df['dem_to_rep_ratio'] <= outliers_below_Q1]).sort_values(
#     by='dem_to_rep_ratio') 
# There is no couties below the first interquartile ranges
(final_df[final_df['dem_to_rep_ratio'] >= outliers_above_Q3]).sort_values(
    by='dem_to_rep_ratio')

In [0]:
f, axes = plt.subplots(1, 2)

outcome_hist = sns.distplot(final_df['dem_to_rep_ratio'] , kde = False, norm_hist = True, ax=axes[0]) # can't see the max value of 22...some more digging is needed

outcome_box = sns.boxplot(final_df['dem_to_rep_ratio'],  orient='v' , ax=axes[1]) # lots of outliers that are 1.5 IQR above the third quartile
plt.suptitle("Distribution of dem_to_rep_ratio")
;

# data appears biased since democrats are in the numerator

# even though there is a lot of skew, we can use a linear regression
# and assume normality due to the central limit theorm

### Log Transforming

In [0]:
# Based on our summary statistics, our max greater than our minimum by a magnitude of 1,000
# Let's see if log transforming does anything

f, axes = plt.subplots(2, 2)

log_dem_to_rep_ratio = np.log(final_df['dem_to_rep_ratio'])

sns.distplot(log_dem_to_rep_ratio, kde = False, norm_hist = True, ax=axes[0, 0])

sns.boxplot(log_dem_to_rep_ratio,  orient='v' , ax=axes[0, 1]) # outliers are on both sides of boxplot 


### NON TRANSFORMED PLOTS REPLOTTED

sns.distplot(final_df['dem_to_rep_ratio'] , kde = False, norm_hist = True, ax=axes[1, 0])

sns.boxplot(final_df['dem_to_rep_ratio'],  orient='v' , ax=axes[1, 1])


plt.suptitle("Distribution of Democratic to Republican Voter Ratio")

axes[:, 0][0].set_ylabel("log transformed", size = 'large')
axes[:, 0][1].set_ylabel("no transformation", size = 'large')

axes[:, 0][0].set_xlabel("log(dem_to_rep_ratio)")
;

In [0]:
final_df['log_ratio'] = np.log(final_df['dem_to_rep_ratio'])

final_df.head()

In [0]:
# the average log of the dem_to_rep_ratio
np.mean(np.log(final_df['dem_to_rep_ratio']))

In [0]:
# counties that are strongly Republican (1.5 IQR under the first quartile)
first_quartile_for_log = np.percentile(final_df['log_ratio'], 25)
third_quartile_for_log = np.percentile(final_df['log_ratio'], 75)
dem_rep_log_iqr = third_quartile_for_log - first_quartile_for_log

print("First Quartile: ", first_quartile_for_log)

log_very_rep = (final_df[final_df['log_ratio'] <= (
    first_quartile_for_log - 1.5*dem_rep_log_iqr)]).sort_values(
        by='log_ratio', ascending=True)

log_very_rep.head()

In [0]:
# counties that are strongly Democratic (1.5 IQR above the third quartile)
print("Third Quartile: ", third_quartile_for_log)

log_very_dem = (final_df[final_df['log_ratio'] >= (
    third_quartile_for_log + 1.5*dem_rep_log_iqr)]).sort_values(
        by='log_ratio', ascending=False)

log_very_dem.head()

In [0]:
## TO DO:
# Create another cloropleth with log transformed ratio

## Feature Assessment

In [0]:
# checking the correlation matrix of our variables

sns.heatmap(final_df[['dem_to_rep_ratio', 'stay at home', '>50 gatherings',
       '>500 gatherings', 'public schools', 'restaurant dine-in',
       'entertainment/gym', 'log_ratio']].corr(), annot=True)

plt.title("Correlation matrix for all 6 original covariates and dem_to_rep_ratio (untransformed) and its log transformation")
;

## top right and bottom left have high correlation because
## they are transformations of each other

## variables have virtually no correlation with direct ratio
## variables have low correlation [0.3, 0.5] with log-transformed
## source: https://www.andrews.edu/~calkins/math/edrm611/edrm05.htm

## probs should stick with the log-transformed version

In [0]:
# indepdent_vars.columns

indepdent_vars = final_df[['stay at home', '>50 gatherings',
       '>500 gatherings', 'public schools', 'restaurant dine-in',
       'entertainment/gym']]

In [0]:
# a function to find each feature's variation inflation factor
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df, drop_cols = None):
    if drop_cols is not None:
        df = df.drop(columns = drop_cols)
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif["features"] = df.columns

    return vif

In [0]:
indepdent_vars = final_df.iloc[:, 5:11]


calculate_vif(indepdent_vars)

In [0]:
calculate_vif(indepdent_vars, '>50 gatherings')

In [0]:
calculate_vif(indepdent_vars, ['>50 gatherings', 'restaurant dine-in'])

In [0]:
# checking our correlation matrix again
sns.heatmap(final_df[['dem_to_rep_ratio', 'stay at home', 
       '>500 gatherings', 'public schools', 
       'entertainment/gym', 'log_ratio']].corr(), annot=True)

plt.title("Correlation matrix for selected 4 covariates and dem_to_rep_ratio (untransformed) and its log transformation")
;

## not a lot of change to the correlation matrix
## indicates that there isn't much loss by excluding them
## some improvement with l

## Making Our Model

In [0]:
# split the data set into a training set and test set

from sklearn.model_selection import train_test_split
np.random.seed(47)

X = final_df[[
    'stay at home', '>50 gatherings', '>500 gatherings', 'public schools',
    'restaurant dine-in', 'entertainment/gym'
]]
Y = pd.Series(final_df['dem_to_rep_ratio'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
#X_train, y_train

In [0]:
# first linear model
import sklearn.linear_model as lm

linear_model = lm.LinearRegression()

# Fit your linear model
linear_model.fit(X_train, Y_train)
Y_pred = linear_model.predict(X_train)

In [0]:
# scatterplot of training set of first linear model

import matplotlib.pyplot as plt
plt.scatter(Y_train, linear_model.predict(X_train), alpha=0.5)
plt.xlabel("Democratic to Republic Ratio of the Training Set")
plt.ylabel("Predicted Democratic to Republic Ratio of the Training Set")
plt.title(
    "Democratic to Republic Ratio vs Predicted Democratic to Republic Ratio of the Training Set"
)

In [0]:
# function to calculate RMSE
def rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted)**2))

In [0]:
# error of our model
train_error = rmse(Y_train, Y_pred)

print("Training RMSE:", train_error) # eh this is OK...will making it log-linear help?

### With Feature Selection

In [0]:
# second training and training sets with select features
X_select = final_df[[
    'stay at home', '>500 gatherings', 'public schools',
    'entertainment/gym'
]]
Y = pd.Series(final_df['dem_to_rep_ratio'])

X_train_select, X_test_select, Y_train_select, Y_test_select = train_test_split(X_select, Y, test_size=0.1)


In [0]:
# second model
select_linear_model = lm.LinearRegression()

# Fit your linear model
select_linear_model.fit(X_train_select, Y_train_select)
Y_pred_select = select_linear_model.predict(X_train_select)

In [0]:
# error of our model
select_train_error = rmse(Y_train_select, Y_pred_select)

print("Selected Feature Training RMSE:", select_train_error)

## With the Log-Transformed Outcome

In [0]:
# log of dem_to_rep_ratio as new Y
log_X = final_df[[
    'stay at home', '>500 gatherings', 'public schools',
    'entertainment/gym'
]]

log_Y = pd.Series(final_df['log_ratio'])

log_X_train, log_X_test, log_Y_train, log_Y_test = train_test_split(log_X, log_Y, test_size=0.1) # 90:10 split since dataset is large
#X_train, y_train

log_X

In [0]:
# third model
log_linear_model = lm.LinearRegression()

# Fit your linear model
log_linear_model.fit(log_X_train, log_Y_train)
log_Y_pred = log_linear_model.predict(log_X_train)

In [0]:
# error of the third model
log_train_error = rmse(log_Y_train, log_Y_pred)

print("Log-Linear Training RMSE:", log_train_error)

In [0]:
print("Training RMSE:", train_error)
print("Selected Feature Training RMSE:", select_train_error)
print("Log-Linear Training RMSE:", log_train_error) # best model!

In [0]:
# scatterplot of our third model
plt.scatter(log_Y_train, log_Y_pred, alpha=0.5)
plt.xlabel("Democratic to Republic Ratio of the Training Set")
plt.ylabel("Predicted Democratic to Republic Ratio of the Training Set")
plt.title(
    "Democratic to Republic Ratio vs Predicted Democratic to Republic Ratio of the Training Set"
)
;

## Predicting Political Party Majority

In [0]:
final_df.loc[final_df['dem_to_rep_ratio'] > 1, 'majority'] = 0 # Democratic Majority = 0
final_df.loc[final_df['dem_to_rep_ratio'] < 1, 'majority'] = 1 # Republican Majority = 1

final_df = final_df.astype({'majority': 'int32'})

In [0]:
final_df = final_df[['countyFIPS', 'CountyName', 'StateName', 'dem_to_rep_ratio',
                      'log_ratio', 'majority',
                      'Date of First Case', 'stay at home', '>50 gatherings',
                      '>500 gatherings', 'public schools', 'restaurant dine-in',
                      'entertainment/gym']]

final_df.head()

In [0]:
# creating the majority column and training and testing sets
logm_X = final_df[['stay at home', '>50 gatherings',
                   '>500 gatherings', 'public schools', 'restaurant dine-in',
                   'entertainment/gym'
]]

logm_Y = pd.Series(final_df['majority'])


logm_X_train, logm_X_test, logm_Y_train, logm_Y_test = train_test_split(logm_X, logm_Y, test_size=0.1) # 90:10 split since dataset is large


In [0]:
# logistic regression
log_model = lm.LogisticRegression()

log_model = log_model.fit(logm_X_train, logm_Y_train)

logm_Y_pred = log_model.predict(logm_X_train)

In [0]:
# accuracy of full logistic model
logisticm_training_accuracy = log_model.score(logm_X_train, logm_Y_train)

print("Full Logistic Model Training Accuracy:", logisticm_training_accuracy)

In [0]:
# creating the majority column and training and testing sets
# NOTE: SAME EXACT VARIABLES AS FULL MODEL
# (didn't have to distinguish since the second model was better anyways
logm_X = final_df[[
    'stay at home', '>500 gatherings', 'public schools',
    'entertainment/gym'
]]

logm_Y = pd.Series(final_df['majority'])


logm_X_train, logm_X_test, logm_Y_train, logm_Y_test = train_test_split(logm_X, logm_Y, test_size=0.1) # 90:10 split since dataset is large


In [0]:
# logistic regression
log_model = lm.LogisticRegression()

log_model = log_model.fit(logm_X_train, logm_Y_train)

logm_Y_pred = log_model.predict(logm_X_train)

In [0]:
# accuracy of selected logistic model
logisticm_training_accuracy = log_model.score(logm_X_train, logm_Y_train)

print("Logistic Model Training Accuracy:", logisticm_training_accuracy)

## the selected logistical model did marginally better than the full model


## Validing Our Model with the Test Data
How well did we do?

### Log-Linear Model

In [0]:
test_error = rmse(log_Y_test, log_linear_model.predict(log_X_test))
print("Test RMSE:", test_error) # want lower error

In [0]:
print(log_linear_model.intercept_)  
print(log_linear_model.coef_)

In [0]:
m1, m2, m3, m4 = log_linear_model.coef_
b = log_linear_model.intercept_
print('dem_to_rep_ratio = {0} + {1} * stay at home + {2} * >500 gatherings + {3} * public schools + {4} * entertainment/gym'.format(round(b, 3), round(m1, 3), round(m2, 3), round(m3, 3), round(m4, 3)))

In [0]:
display(
    Latex(
        r'\text{{log}}(\texttt{{dem\_to\_rep\_ratio}}) = {0} + {1} * \texttt{{stay at home}} + {2} * \texttt{{>500 gatherings}} + {3} * \texttt{{public schools}} + {4} * \texttt{{entertainment/gym}}'.format(round(b, 3), round(m1, 3), round(m2, 3), round(m3, 3), round(m4, 3))
    )
)

### Logistic Model

In [0]:
logm_accuracy = log_model.score(logm_X_test, logm_Y_test)
print("Test Accuracy:", logm_accuracy) # want higher accuracy


In [0]:
print(log_model.intercept_)  
print(log_model.coef_)

In [0]:
m1, m2, m3, m4 = log_model.coef_[0]
b = log_model.intercept_[0]

In [0]:
display(
    Latex(
        r'\text{{logit}}(\texttt{{majority}}) = {0} + {1} * \texttt{{stay at home}} + {2} * \texttt{{>500 gatherings}} + {3} * \texttt{{public schools}} + {4} * \texttt{{entertainment/gym}}'.format(round(b, 3), round(m1, 3), round(m2, 3), round(m3, 3), round(m4, 3))
    )
)

## Data Visualization

### Population

In [0]:
import plotly.express as px
import json

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# # if file is local
# counties = json.dumps('./geojson-counties-fips.json')

In [0]:
fig_log_party = px.choropleth_mapbox(final_df, geojson=counties, locations='countyFIPS', color='log_ratio',
                           color_continuous_scale="bluered_r",
                           range_color=(-4, 4),
                           mapbox_style="carto-positron",
                           hover_name = final_df['CountyName'],
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'log_ratio':'log(Dem:Rep Ratio)'}, 
                           title = 'log(Democrat to Republican Ratio) of US Counties in 2016 Election'
                          )


fig_log_party.update_layout(margin={"r":0,"l":0,"b":0})
fig_log_party.show()

In [0]:
fig_majority = px.choropleth_mapbox(final_df.replace({'majority' : {0 : "Democratic", 1 : "Republican"}}), geojson=counties, locations='countyFIPS', color='majority',
                        #    color_continuous_scale="bluered",
                           color_discrete_map = {"Democratic" : "#0000ff", "Republican" : "#ff0000"},
                        #    range_color=(0, 1),
                           mapbox_style="carto-positron",
                           hover_name = final_df['CountyName'],
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                        #    labels={'log_ratio':'log(Dem:Rep Ratio)'}, 
                           title = 'Political Party Majority of US Counties in 2016 Election'
                          )


fig_majority.update_layout(margin={"r":0,"l":0,"b":0}, coloraxis_showscale=True, showlegend=True, geo = dict(showsubunits=True))

fig_majority.show()

In [0]:
df['log_pop'] = np.log(df['PopulationEstimate2018'])

fig_pop = px.choropleth_mapbox(df, geojson=counties, locations='countyFIPS', color='log_pop',
                           color_continuous_scale="Viridis",
                           range_color=(0, 13),
                           mapbox_style="carto-positron",
                           hover_name = df['CountyName'],
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'log_pop':'log(Population)'}, 
                           title = 'Population of US Counties in 2018'
                          )


fig_pop.update_layout(margin={"r":0,"l":0,"b":0})
fig_pop.show()